Available Captures for Rook. 棋盘上车能吃的棋子数。8*8的国际棋盘，有一个白车(R)，若干白象(B)和黑卒(p)，求白车当前捕获到的对方旗子数量。

In [24]:
# 思路：上下左右四个方位必须直线相通才能捕获到，并且不能被B阻拦
# 输入字符串化，在行内判断是否存在'pR'或'Rp'
# 在列的方向上判断是否存在'pR'或'Rp'


def numRookCaptures(board):
    res = 0
    # 首先索引到R的坐标
    for row in board:
        if 'R' in row:
            tmp = ''.join(row).replace('.','')
            res += 1 if 'pR' in tmp else 0
            res += 1 if 'Rp' in tmp else 0

            col_idx = row.index('R')
            break

    # 将'R'所在的一列提取出来
    col = [row[col_idx] for row in board]
    col = ''.join(col).replace('.','')
    res += 1 if 'pR' in col else 0
    res += 1 if 'Rp' in col else 0

    return res

[Game of Life](https://leetcode.com/problems/game-of-life/)。给定一个细胞矩阵，0代表死细胞，1代表活细胞。细胞的状态转移规则如下：
- 周围活细胞数目为2或3则该细胞正常存活
- 周围活细胞数目小于2则该细胞死亡
- 周围活细胞数目大于3则该细胞死亡
- 周围活细胞数目等于3则该细胞复活

In [3]:
# 思路：程序实现时是逐行逐列扫描并改变矩阵值的，那么怎么保存转换之前的转态是关键
# 使用状态码，不难得出有四种可能的状态：
# 0.死细胞变死细胞；1.活细胞变死细胞；2.死细胞变活细胞；3.活细胞变活细胞
# 注意这里状态码的设置有一个技巧，状态码模2为转换前的存活状态，状态码整除2为转换后的存活状态
# 并且模2运算的状态与初始状态相同
# 扫描矩阵两次，第一次做模运算设置状态码，第二次做整除运算设置存活状态


def gameOfLife(board) -> None:
    rows, cols = len(board), len(board[0])
    delta = list(zip([-1, -1, -1, 0, 1, 1, 1, 0], [-1, 0, 1, 1, 1, 0, -1, -1]))

    def check_staus(row, col):
        live_nb = 0    # 转换前存活的邻居数

        for dx, dy in delta:
            x = row + dx
            y = col + dy
            if 0 <= x < rows and 0 <= y < cols and board[x][y] % 2 == 1:
                live_nb += 1

        if board[row][col] % 2 == 0 and live_nb == 3:    # 死细胞复活
            return 2
        elif board[row][col] % 2 == 1:    # 之前为活细胞
            if live_nb == 2 or live_nb == 3:    # 存活
                return 3
            else:    # 死亡
                return 1
        else:
            return 0

    # 设置状态码
    for row in range(rows):
        for col in range(cols):
            board[row][col] = check_staus(row, col)

    # 根据状态码设置存活状态
    for row in range(rows):
        for col in range(cols):
            board[row][col] = board[row][col]//2

[[0, 0, 0], [1, 0, 1], [0, 1, 1], [0, 1, 0]]